In [51]:
import pandas as pd
from datetime import date, timedelta

from app import get_last_thursday

urlCatTRAB = "https://docs.google.com/spreadsheets/d/1f1l2OFLYFqWNcy084IiATyquMH7v2nnRx3lKfE8QAH0/gviz/tq?tqx=out:csv&sheet=catTRAB"
urlMEZCLADO = "https://docs.google.com/spreadsheets/d/1fzy0h-g0-LbRxNcURZJqyGuIZOoJHLFkQDZ5vpAb4zc/gviz/tq?tqx=out:csv&sheet=MEZCLADO"
urlLAMINADO = "https://docs.google.com/spreadsheets/d/1fzy0h-g0-LbRxNcURZJqyGuIZOoJHLFkQDZ5vpAb4zc/gviz/tq?tqx=out:csv&sheet=LAMINADO"
urlFORMULADO = "https://docs.google.com/spreadsheets/d/1fzy0h-g0-LbRxNcURZJqyGuIZOoJHLFkQDZ5vpAb4zc/gviz/tq?tqx=out:csv&sheet=FORMULADO"
urlVULCANIZADO = "https://docs.google.com/spreadsheets/d/1fzy0h-g0-LbRxNcURZJqyGuIZOoJHLFkQDZ5vpAb4zc/gviz/tq?tqx=out:csv&sheet=VULCANIZADO"
urlCARDADO = "https://docs.google.com/spreadsheets/d/1fzy0h-g0-LbRxNcURZJqyGuIZOoJHLFkQDZ5vpAb4zc/gviz/tq?tqx=out:csv&sheet=CARDADO"

CATTRAB = pd.read_csv(urlCatTRAB)
MEZCLADO = pd.read_csv(urlMEZCLADO)
LAMINADO = pd.read_csv(urlLAMINADO)
FORMULADO = pd.read_csv(urlFORMULADO)
VULCANIZADO = pd.read_csv(urlVULCANIZADO)
CARDADO = pd.read_csv(urlCARDADO)

last_thursday = get_last_thursday()
today = date.today()

today = today.strftime("%Y/%m/%d")
last_thursday = last_thursday.strftime("%Y/%m/%d")

MEZCLADO["Fecha"] = pd.to_datetime(MEZCLADO["Fecha"], dayfirst=True)
LAMINADO["Fecha"] = pd.to_datetime(LAMINADO["Fecha"], dayfirst=True)
FORMULADO["Fecha"] = pd.to_datetime(FORMULADO["Fecha"], dayfirst=True)
VULCANIZADO["Fecha"] = pd.to_datetime(VULCANIZADO["Fecha"], dayfirst=True)
CARDADO["Fecha"] = pd.to_datetime(CARDADO["Fecha"], dayfirst=True)
LAMINADO["IdTrabajador02"] = LAMINADO["IdTrabajador02"].astype(int)
FORMULADO["idTrabajador02"] = pd.array(FORMULADO["idTrabajador02"])

In [52]:
FILTERED_FORMULADO = FORMULADO[FORMULADO["Fecha"].isin(
    pd.date_range(last_thursday, today))]
MERGED_FORMULADO = FILTERED_FORMULADO.merge(
    CATTRAB, on=["idTrabajador"], how="right")

# Create a dictionary to store the indices for each 'idTrabajador02'
formulado_indices = {}

# Iterate over the rows of 'MERGED_FORMULADO' and populate the indices_dict
for index, row in MERGED_FORMULADO.iterrows():
    if pd.notna(row['Fecha']):
        id_trabajador_01 = row["idTrabajador"]
        id_trabajador_02 = row['idTrabajador02']
        fecha = row['Fecha']

        formulado_indices[int(id_trabajador_01)] = fecha

        # Skip if 'idTrabajador02' is NaN
        if pd.isna(id_trabajador_02):
            continue

        if id_trabajador_01 in formulado_indices:
            formulado_indices[int(id_trabajador_01)].append(fecha)
        else:
            formulado_indices[int(id_trabajador_01)] = [fecha]

        # Add the index to the list of indices for 'idTrabajador02'
        for id in id_trabajador_02.split(','):
            formulado_indices[int(id)] = fecha

            if pd.notna(id_trabajador_02) and id_trabajador_02 in formulado_indices:
                formulado_indices[int(id_trabajador_02)].append(fecha)
            elif pd.notna(id_trabajador_02):
                formulado_indices[int(id_trabajador_02)] = [fecha]

{}

In [53]:
FILTERED_MEZCLADO = MEZCLADO[MEZCLADO["Fecha"].isin(
    pd.date_range(last_thursday, today))]

MERGED_MEZCLADO = FILTERED_MEZCLADO.merge(
    CATTRAB, on=["idTrabajador"], how="right")

mezclado_indices = {}

for index, row in MERGED_MEZCLADO.iterrows():
    if pd.notna(row['Fecha']):
        id_trabajador_01 = row['idTrabajador']
        fecha = row['Fecha']
        mezclado_indices[int(id_trabajador_01)] = fecha

{}

In [54]:
FILTERED_LAMINADO = LAMINADO[LAMINADO["Fecha"].isin(
    pd.date_range(last_thursday, today))]
data = FILTERED_LAMINADO.merge(CATTRAB, on=["idTrabajador"], how="right")

ayudantes = data[data['IdTrabajador02'].notna()]
ayudantes = ayudantes[['IdTrabajador02', 'Fecha']].copy()
ayudantes.columns = ['idTrabajador', 'Fecha_Ayudante']
MERGED_LAMINADO = pd.merge(
    data, ayudantes, how='left', left_on='idTrabajador', right_on='idTrabajador')
MERGED_LAMINADO.loc[MERGED_LAMINADO['Fecha_Ayudante'].notna(
), 'Fecha'] = MERGED_LAMINADO['Fecha_Ayudante']

laminado_indices = {}

for index, row in MERGED_LAMINADO.iterrows():
    if pd.notna(row['Fecha']):
        id_trabajador_01 = row["idTrabajador"]
        id_trabajador_02 = row['IdTrabajador02']
        fecha = row['Fecha']
        
        if id_trabajador_01 in laminado_indices:
            laminado_indices[int(id_trabajador_01)].append(fecha)
        else:
            laminado_indices[int(id_trabajador_01)] = [fecha]

        if pd.notna(id_trabajador_02) and id_trabajador_02 in laminado_indices:
            laminado_indices[int(id_trabajador_02)].append(fecha)
        elif pd.notna(id_trabajador_02):
            laminado_indices[int(id_trabajador_02)] = [fecha]

{11: [Timestamp('2023-07-21 00:00:00')],
 23: [Timestamp('2023-07-21 00:00:00'), Timestamp('2023-07-21 00:00:00')],
 42: [Timestamp('2023-07-21 00:00:00'), Timestamp('2023-07-25 00:00:00')]}

In [55]:
FILTERED_VULCANIZADO = VULCANIZADO[VULCANIZADO["Fecha"].isin(
    pd.date_range(last_thursday, today))]
data = FILTERED_VULCANIZADO.merge(
    CATTRAB, on=["idTrabajador"], how="right")

ayudantes = data[data['idTrabajador02'].notna()]
ayudantes = ayudantes[['idTrabajador02', 'Fecha']].copy()
ayudantes.columns = ['idTrabajador', 'Fecha_Ayudante']
MERGED_VULCANIZADO = pd.merge(
    data, ayudantes, how='left', left_on='idTrabajador', right_on='idTrabajador')
MERGED_VULCANIZADO.loc[MERGED_VULCANIZADO['Fecha_Ayudante'].notna(
), 'Fecha'] = MERGED_VULCANIZADO['Fecha_Ayudante']

vulcanizado_indices = {}

for index, row in MERGED_VULCANIZADO.iterrows():
    if pd.notna(row['Fecha']):
        id_trabajador_01 = row["idTrabajador"]
        id_trabajador_02 = row['idTrabajador02']
        fecha = row['Fecha']

        if id_trabajador_01 in vulcanizado_indices:
            vulcanizado_indices[int(id_trabajador_01)].append(fecha)
        else:
            vulcanizado_indices[int(id_trabajador_01)] = [fecha]

        if pd.notna(id_trabajador_02) and id_trabajador_02 in vulcanizado_indices:
            vulcanizado_indices[int(id_trabajador_02)].append(fecha)
        elif pd.notna(id_trabajador_02):
            vulcanizado_indices[int(id_trabajador_02)] = [fecha]

{16: [Timestamp('2023-07-20 00:00:00'),
  Timestamp('2023-07-20 00:00:00'),
  Timestamp('2023-07-20 00:00:00'),
  Timestamp('2023-07-21 00:00:00'),
  Timestamp('2023-07-21 00:00:00'),
  Timestamp('2023-07-20 00:00:00'),
  Timestamp('2023-07-20 00:00:00'),
  Timestamp('2023-07-20 00:00:00'),
  Timestamp('2023-07-21 00:00:00'),
  Timestamp('2023-07-21 00:00:00')],
 52: [Timestamp('2023-07-20 00:00:00'),
  Timestamp('2023-07-20 00:00:00'),
  Timestamp('2023-07-20 00:00:00'),
  Timestamp('2023-07-21 00:00:00'),
  Timestamp('2023-07-21 00:00:00')]}

In [56]:
FILTERED_CARDADO = CARDADO[CARDADO["Fecha"].isin(
    pd.date_range(last_thursday, today))]
data = FILTERED_CARDADO.merge(CATTRAB, on=["idTrabajador"], how="right")

ayudantes = data[data['idTrabajador02'].notna()]
ayudantes = ayudantes[['idTrabajador02', 'Fecha']].copy()
ayudantes.columns = ['idTrabajador', 'Fecha_Ayudante']
MERGED_CARDADO = pd.merge(
    data, ayudantes, how='left', left_on='idTrabajador', right_on='idTrabajador')
MERGED_CARDADO.loc[MERGED_CARDADO['Fecha_Ayudante'].notna(
), 'Fecha'] = MERGED_CARDADO['Fecha_Ayudante']

cardado_indices = {}

for index, row in MERGED_CARDADO.iterrows():
    if pd.notna(row['Fecha']):
        id_trabajador_01 = row["idTrabajador"]
        id_trabajador_02 = row['idTrabajador02']
        fecha = row['Fecha']

        if id_trabajador_01 in cardado_indices:
            cardado_indices[int(id_trabajador_01)].append(fecha)
        else:
            cardado_indices[int(id_trabajador_01)] = [fecha]

        if pd.notna(id_trabajador_02) and id_trabajador_02 in cardado_indices:
            cardado_indices[int(id_trabajador_02)].append(fecha)
        elif pd.notna(id_trabajador_02):
            cardado_indices[int(id_trabajador_02)] = [fecha]

{2: [Timestamp('2023-07-20 00:00:00'),
  Timestamp('2023-07-20 00:00:00'),
  Timestamp('2023-07-21 00:00:00'),
  Timestamp('2023-07-21 00:00:00'),
  Timestamp('2023-07-21 00:00:00'),
  Timestamp('2023-07-21 00:00:00'),
  Timestamp('2023-07-22 00:00:00'),
  Timestamp('2023-07-22 00:00:00'),
  Timestamp('2023-07-22 00:00:00'),
  Timestamp('2023-07-24 00:00:00'),
  Timestamp('2023-07-24 00:00:00'),
  Timestamp('2023-07-24 00:00:00')],
 14: [Timestamp('2023-07-20 00:00:00'),
  Timestamp('2023-07-20 00:00:00'),
  Timestamp('2023-07-21 00:00:00'),
  Timestamp('2023-07-21 00:00:00'),
  Timestamp('2023-07-21 00:00:00'),
  Timestamp('2023-07-21 00:00:00'),
  Timestamp('2023-07-22 00:00:00'),
  Timestamp('2023-07-22 00:00:00'),
  Timestamp('2023-07-22 00:00:00'),
  Timestamp('2023-07-24 00:00:00'),
  Timestamp('2023-07-24 00:00:00'),
  Timestamp('2023-07-24 00:00:00'),
  Timestamp('2023-07-20 00:00:00'),
  Timestamp('2023-07-20 00:00:00'),
  Timestamp('2023-07-21 00:00:00'),
  Timestamp('2023-07

In [57]:
# Combine all dictionaries
combined_dict = {}
dictionaries = [vulcanizado_indices, laminado_indices,
                mezclado_indices, formulado_indices, cardado_indices]

# Merge dictionaries
for dictionary in dictionaries:
    for key, value in dictionary.items():
        for date in set(value):
            if key in combined_dict:
                combined_dict[key].append(date)
            else:
                combined_dict[key] = [date]

for key, value in combined_dict.items():
    combined_dict[key] = set(value)

combined_dict

{16: {Timestamp('2023-07-20 00:00:00'), Timestamp('2023-07-21 00:00:00')},
 52: {Timestamp('2023-07-20 00:00:00'), Timestamp('2023-07-21 00:00:00')},
 11: {Timestamp('2023-07-21 00:00:00')},
 23: {Timestamp('2023-07-21 00:00:00')},
 42: {Timestamp('2023-07-21 00:00:00'), Timestamp('2023-07-25 00:00:00')},
 2: {Timestamp('2023-07-20 00:00:00'),
  Timestamp('2023-07-21 00:00:00'),
  Timestamp('2023-07-22 00:00:00'),
  Timestamp('2023-07-24 00:00:00')},
 14: {Timestamp('2023-07-20 00:00:00'),
  Timestamp('2023-07-21 00:00:00'),
  Timestamp('2023-07-22 00:00:00'),
  Timestamp('2023-07-24 00:00:00')}}